In [ ]:
!pip install git+https://github.com/tky823/ssspy.git@develop

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_dummy_data

In [ ]:
n_sources = 3
max_samples = 10 * 16000
sisec2010_tag = "dev1_female3"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img = download_dummy_data(n_sources=n_sources, sisec2010_tag=sisec2010_tag, max_samples=max_samples) # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
from ssspy.transform import whiten
from ssspy.algorithm import projection_back
from ssspy.bss.iva import NaturalGradIVA

In [ ]:
def contrast_fn(y):
    return 2 * np.linalg.norm(y, axis=1)

def score_fn(y):
    norm = np.linalg.norm(y, axis=1, keepdims=True)
    norm = np.maximum(norm, 1e-10)
    return y / norm

## Holonomic type

In [ ]:
natural_grad_iva = NaturalGradIVA(
    step_size=1e-1,
    contrast_fn=contrast_fn,
    score_fn=score_fn,
    is_holonomic=True
)
print(natural_grad_iva)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = natural_grad_iva(spectrogram_mix, n_iter=500)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
plt.figure()
plt.plot(natural_grad_iva.loss)
plt.show()
plt.close()

## Non-holonomic type

In [ ]:
natural_grad_iva = NaturalGradIVA(
    step_size=1e+1,
    contrast_fn=contrast_fn,
    score_fn=score_fn,
    is_holonomic=False,
    should_apply_projection_back=False
)
print(natural_grad_iva)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_mix_whitened = whiten(spectrogram_mix)
spectrogram_est = natural_grad_iva(spectrogram_mix_whitened, n_iter=500)
spectrogram_est = projection_back(spectrogram_est, reference=spectrogram_mix)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for waveform in waveform_est:
    display(ipd.Audio(waveform, rate=16000))

In [ ]:
plt.figure()
plt.plot(natural_grad_iva.loss)
plt.show()
plt.close()